# Contoh Dasar AutoGen

Dalam contoh kode ini, Anda akan menggunakan Kerangka Kerja AI [AutoGen](https://aka.ms/ai-agents/autogen) untuk membuat agen dasar.

Tujuan dari contoh ini adalah untuk menunjukkan langkah-langkah yang nantinya akan kita gunakan dalam contoh kode tambahan saat mengimplementasikan berbagai pola agen.


## Impor Paket Python yang Dibutuhkan


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Membuat Klien

Dalam contoh ini, kita akan menggunakan [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) untuk mengakses LLM.

`model` didefinisikan sebagai `gpt-4o-mini`. Cobalah mengganti model dengan model lain yang tersedia di marketplace GitHub Models untuk melihat hasil yang berbeda.

Sebagai uji coba cepat, kita akan menjalankan prompt sederhana - `Apa ibu kota Prancis`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Mendefinisikan Agen

Sekarang setelah kita mengatur `client` dan memastikan bahwa itu berfungsi, mari kita buat sebuah `AssistantAgent`. Setiap agen dapat diberikan:  
**name** - Nama pendek yang berguna untuk merujuknya dalam alur multi-agen.  
**model_client** - Klien yang telah Anda buat pada langkah sebelumnya.  
**tools** - Alat-alat yang tersedia yang dapat digunakan Agen untuk menyelesaikan tugas.  
**system_message** - Metaprompt yang mendefinisikan tugas, perilaku, dan nada dari LLM.  

Anda dapat mengubah system message untuk melihat bagaimana LLM merespons. Kita akan membahas `tools` di Pelajaran #4.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Jalankan Agen

Fungsi di bawah ini akan menjalankan agen. Kami menggunakan metode `on_message` untuk memperbarui status Agen dengan pesan baru.

Dalam kasus ini, kami memperbarui status dengan pesan baru dari pengguna yaitu `"Rencanakan liburan cerah yang menyenangkan untuk saya"`.

Anda dapat mengubah isi pesan untuk melihat bagaimana LLM merespons secara berbeda.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan penerjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berusaha untuk memberikan hasil yang akurat, harap diingat bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang otoritatif. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
